In [21]:
import pandas as pd
import numpy as np
import pdfquery
import json
from shapely.geometry import box
from shapely.ops import cascaded_union
import pdftableextract as pte
from math import floor
import pickle
from pymongo import MongoClient
from os.path import isfile, join, isdir
from os import listdir
import re
import hashlib
import unidecode
from fuzzysearch import find_near_matches
import textract
import unicodedata
class dbConf(object):
    # connect to database
    client = MongoClient('mongodb://localhost:27017/')
    try:
        mdb = client.racmo
        Notifications = mdb.Notifications
        PdfFiles=mdb.PdfFiles
        Keywords=mdb.Keywords
    except Exception as e:
        print(e)

PDF_DIR = "/home/thrymr/Racmo/RacmoIT/process/Gestured documents 15-02-2018"


In [22]:
with open('../RacmoIT.pickle', 'rb') as handle:
    b = pickle.load(handle)

In [23]:
# kdf=b['keywords']
fdf=b['documents']

In [83]:
with open('Final_Keyword_Analysis.pickle', 'rb') as handle:
    b=pickle.load(handle)
kxddf=b['keywordsX']
# fdf=fdf.rename(columns={'file':'filename','type':'filetype','group':'filegroup'})
kxddf[kxddf['keyword'].apply(lambda x : 'EJECUCION DE TITULOS NO JUDICIALES'.lower() in x)]

,fileclass,filetype,purpose,notification_type,keyword,bias


In [89]:
# fgs=fdf.groupby('filegroup')
# fgdf=pd.DataFrame(columns=['filegroup'])
# i=0
# fgdf['files']=np.empty((293, 0)).tolist()
# fgdf['filetypes']=np.empty((293, 0)).tolist()
# for k,v in fgs.groups.items():
    
#     fgdf.loc[i,'filegroup']=k
#     files=[]
#     pcs=[]
#     filetypes=[]
#     for ind in v:
#             files.append(fdf.loc[ind,'filename'])
#             filetypes.append(fdf.loc[ind,'filetype'])
#     fgdf.loc[i,'filetypes']=filetypes
#     fgdf.loc[i,'files']=files
#     i+=1
# fgdf          

In [85]:
c=0
for i ,r in fdf[~(fdf['table_response'].str.contains('Error:'))&(fdf['filetype']=='TICKET')].iterrows():
    if r['table_response'][:5]!='Error':
        js=json.loads(r['table_response'])
        pf=r['filename'].split('.')[0]+'_'+''.join(js['table_1']['Documentos'][0][0].split()).split('(Principal)')[0]
        fgf=[''.join(x.split())for x in fgdf.loc[fgdf['filegroup']==r['filegroup'],'files'].values[0] ]
        if pf in fgf:
            fl=fgdf.loc[fgdf['filegroup']==r['filegroup'],'files'].values[0][fgf.index(pf)]
            fdf.loc[fdf['filename'].str.contains(fl),'filetype']="NOTIFICATION"
            c+=1
print c


In [27]:
kdf=kxddf

In [28]:
pdfclass={'N1':'ADMISSION','N2':'TRANSFER OF LEGAL REPRESENTATION PERMITTED',
           'N3':'ENFORCEMENT ORDERS','N4':'STATEMENT OF PAYMENT',
           'N5':'REJECTED CLAIMS','N6':'REJECTED TRANSFER OF LEGAL REPRESENTATION',
           'N7':'HEARING','N8':'ASSET INQUIRY',
           'N9':'PLACE OF RESIDENCY REQUIRED',
           'N10':'REQUIREMENT'}

In [29]:
n=kdf.iterrows()
for i,row in n:
    if 'EJECUCION DE TITULOS NO JUDICIALES'.lower() in row['keyword'] or 'ejecucion de titulos judiciales' in row['keyword']:
        kdf=kdf.drop(kdf.index[i])
        kxddf=kdf.drop(kdf.index[i])
        print i
    elif(not ( row['fileclass'] =='N5'or row['fileclass'] =='N6')):
        if(pdfclass[row['fileclass']] in row['notification_type']):
            
            kdf.loc[i,'purpose']='CLASSIFICATION'
        else:
            kdf.loc[i,'purpose']='EXTRACTION'
    elif row['fileclass'] =='N5':
        if('N5 - REJECTED CLAIMS' in row['notification_type']):
            kdf.loc[i,'purpose']='CLASSIFICATION'
        else:
            kdf.loc[i,'purpose']='EXTRACTION'
    elif row['fileclass'] =='N6':
        if('N6 - REJECTED TRANSFER OF LEGAL REPRESENTATION' in row['notification_type']):
            kdf.loc[i,'purpose']='CLASSIFICATION'
        else:
            kdf.loc[i,'purpose']='EXTRACTION'


173


In [31]:
for i,r in fdf.iterrows():
    try:
        newtext =unicode(textract.process(join(PDF_DIR,r['filename'])),'utf-8')
        newtext=(''.join((c for c in unicodedata.normalize('NFD', newtext) if unicodedata.category(c) != 'Mn'))).lower()
        rem=''
        paratlist=['MODO DE IMPUGNACION:'.lower(),'mode d\'impugnacio','recurso de repelacion','recurs de reposicio','recurso de reposicion','recurso de apelacion']
        if (r['filetype']=='NOTIFICATION') :
            for parat in paratlist:
                if (parat.lower() in newtext) :
                    rem=re.split(r'\s(?=(?:y firmo|y firma))',newtext.split(parat.lower())[-1],1)[0]
            newtext=newtext.replace(rem,'')
    except Exception as e:
        newtext='Error:'+str(e)
    fdf.loc[i,'text_response']=newtext

In [88]:
import re
kdf["document_list"]=np.empty((len(kdf), 0)).tolist()
fdf["keywords"]=fdf["keywords"].apply(lambda x:{})

notextset=set()
for j,row in fdf.iterrows():

    if j%595==0:
        print j
    for i,kdrow in kdf.iterrows():
        text=''.join(row['text_response'].split())
#         
        if row['text_response'][:5]!='Error':
            
            f=True
            for k in kdrow['keyword']:
#                 if not re.search(k, row['text_response'], re.IGNORECASE):
                
#                 print(unidecode.unidecode(k).lower(),find_near_matches(unidecode.unidecode(k).lower(), text, max_l_dist=1))
#                 match=[]
#                 try:
#                     match=find_near_matches(str(unidecode.unidecode(k)).lower(), text, max_l_dist=2)
#                 except Exception as e:
#                     notextset.add(j)
#                 if len(match)==0:
#                     f=False
                if not (''.join(unidecode.unidecode(k).split()).lower() in text):
                    f=False
            if f and (kdrow['filetype']==row['filetype'])and (kdrow['purpose']=='CLASSIFICATION'):
                kdf.loc[i,"document_list"].append(row['hasid'])
                
                if kdrow['fileclass'] in fdf.loc[j,'keywords'].keys():
                    fdf.loc[j,'keywords'][kdrow['fileclass']].append(kdrow['keyword'])
                else:
                    fdf.loc[j,'keywords'][kdrow['fileclass']]=list()
                    fdf.loc[j,'keywords'][kdrow['fileclass']].append(kdrow['keyword'])
# fdf
# for idf,rowdf in fdf.iterrows():
#     keywords=[]
#     keywordclass=[]
    
#     for ki, kr in kddf.iterrows():
#         if rowdf['hasid'] in kr['document_list'] :
#             if kr['file_class'] in fdf.columns:
#                 dokydf.loc[idf,kr['file_class']]+=1
#             else:
#                 dokydf[kr['file_class']]=0
#                 dokydf.loc[idf,kr['file_class']]=1
# .to_csv("keywords.csv")  
# keyrpo=pd.concat([keyrpo.drop(['key_count'], axis=1), keyrpo['key_count'].apply(pd.Series)], axis=1)


In [33]:
fdf['fileclass']=fdf['filename'].apply(lambda x : x.split()[0])
# kdf.to_csv('testKey.csv')
a = {'documents':fdf,'keywords':kdf}
with open('Test.pickle', 'wb') as handle:
    pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [34]:
dokydf=fdf.copy()[['filename','filetype','keywords']]
keyrpodf=kdf.copy()
for idf,rowdf in fdf.iterrows():
    keywords=[]
    keywordclass=[]
    for k,v in rowdf['keywords'].items():
        if 'key '+k in dokydf.columns:
                dokydf.loc[idf,'key '+k].append(v)
        else:
                dokydf['key '+k]=np.empty((len(dokydf), 0)).tolist()
                dokydf.loc[idf,'key '+k].append(v)
    for ki, kr in kdf.iterrows():
            if rowdf['hasid'] in kr['document_list'] :
                if kr['fileclass'] in dokydf.columns:
                    dokydf.loc[idf,kr['fileclass']]+=1
                else:
                    dokydf[kr['fileclass']]=0
                    dokydf.loc[idf,kr['fileclass']]=1
                if 'Filename '+ rowdf['fileclass'] in keyrpodf.columns:
                    keyrpodf.loc[ki,'Filename '+ rowdf['fileclass']].append(rowdf['filename'])
                else:
                    keyrpodf['Filename '+ rowdf['fileclass']]=np.empty((len(keyrpodf), 0)).tolist()
                    keyrpodf.loc[ki,'Filename '+ rowdf['fileclass']].append(rowdf['filename'])
        

# .to_csv("keywords.csv")  
# keyrpo=pd.concat([keyrpo.drop(['key_count'], axis=1), keyrpo['key_count'].apply(pd.Series)], axis=1)

In [35]:
ddf=fdf.copy()[['filename','filegroup','hasid','fileclass','filetype']]
for idf,rowdf in ddf.iterrows():
    for ki, kr in kxddf.iterrows():
        if rowdf['hasid'] in kr['document_list'] :
            c=1
            if not kr['fileclass']==kr['bias'] :
#                 c=kr[kr['file_class']]
                c=0
            if kr['fileclass'] in ddf.columns:
                ddf.loc[idf,kr['fileclass']]+=c
            else:
                ddf[kr['fileclass']]=0
                ddf.loc[idf,kr['fileclass']]=c
print("Part2")

for i,row in ddf.iterrows():
    keyscore=dict(row[['N1','N2','N3','N4','N5','N6','N7','N8','N9','N10']])
    predicted=max(keyscore, key=keyscore.get)
    if row['N5']>0:
        ddf.loc[i,'predicted_class']='N5'
    elif row['N2']>0 and row['N4']>0:
        ddf.loc[i,'predicted_class']='N2+N4'
    elif row['N1']>0 and(row['N10']>0 or row['N9']>0):
        ddf.loc[i,'predicted_class']='N1'
    elif row['N10']>0 and (row['N2']>0 or row['N6']>0):
        ddf.loc[i,'predicted_class']='N2+N4'
    elif keyscore[predicted]>0:
        ddf.loc[i,'predicted_class']=predicted
ddf

KeyError: 'document_list'

In [ ]:
fgs=ddf.groupby('filegroup')
i=0
fgdf['predicted_classes']=np.empty((len(fgdf), 0)).tolist()
for k,v in fgs.groups.items():
    fgdf.loc[i,'filegroup']=k
    pcs=[]
    for ind in v:
            pcs.append(ddf.loc[ind,'predicted_class'])
    fgdf.loc[i,'predicted_classes']=pcs
    i+=1
fgdf          

In [ ]:
# y_actufg=[]
y_predfg=[]
conflic=[]
for k,v in fgs.groups.items():
#     y_actufg.append(ddf.loc[v[0],'fileclass'])
    s=set()
    e='_alpha'
    for i in v:
        s.add(ddf.loc[i,'predicted_class'])
#     s={x for x in s if x==x}
    if(len(s)==1):
        if list(s)[0]==list(s)[0]:
            e=list(s)[0]
            y_predfg.append(list(s)[0])
    else:
        
        if(len({x for x in s if x==x})>1):
            e='Conflict'
             
        else:
            
            for x in list(s):
                if x==x:
                    e=x
    if e!='_alpha' and e!='Conflict':
        fgdf.loc[(fgdf['filegroup'] == k),'group_predicted_class']=e
    elif e=='Conflict':
        for j in v:
            if ddf.loc[j,'filetype']=='TICKET':
                e=ddf.loc[j,'predicted_class']
                fgdf.loc[(fgdf['filegroup'] == k),'group_predicted_class']=e
#     y_predfg.append(e)        
        
        

In [ ]:
a = {'documents':fdf,'keywords':kdf,'filegroup':fgdf}
with open('FinalTest.pickle', 'wb') as handle:
    pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [45]:

with open('FinalTest.pickle', 'rb') as handle:
    a=pickle.load(handle)
fgdf=a['filegroup']
fdf=a['documents']
kdf=a['keywords']

Encoding Decoding

In [46]:
procedures={'CAM':'Cambiario','COG':'Cognicion','CON':'Concurso Acreedores','EJE':'Juicio Ejecutivo','ETJ':'EJECUCION DE TITULOS JUDICIAL','ETJH':'ETJ Continuacion Hipotecario','ETN':'EJECUCION DE TITULOS NO JUDICIAL',\
            'HIP':'EJECUCION Hipotecaria','MCU':'Menor Cuantia','MON':'Monitorio','ORD':'‘Concurso Ordinario','TDO':'Terceria De Domino','TMD':'Terceria De Mejor Derecho','VER':'Verbal','PEN':'Penal','RAP':'Recurso De Apelacion',\
            'PTC':'Pieza Tasacion Costas','PSO':'Pieza Seperada Oposicion','AUX':'Auxilio Nacional','CNJ':'Cosignacion Judicial','RCS':'Recurso De Casacion','INC':'Incidente Concursal','MC':'Medidas Cautelares','CN':'Conciliacion'}
numspan={"1":"uno","2":"dos","3":"tres","4":"cuatro","5":"cinco","6":"seis","7":"siete","8":"ocho","9":"nueve","10":"diez","11":"once","12":"doce","13":"trece","14":"catorce","15":"quince","16":"dieciseis","17" :"diecisiete","18":"dieciocho","19":"diecinueve","20" : "veinte","21" : "veintiuno","22"  :"veintidós","23" : "veintitrés","24" : "veinticuatro","25" : "veinticinco","26":"veintiséis","27" :"veintisiete","28": "veintiocho","29"  :"veintinueve","30" : "treinta"}
numcat={"1":"un","2":"dos","3":"tres","4":"quatre","5":"cinc","6":"sis","7":"set","8":"vuit","9":"nou","10":"deu","11":"onze","12":"dotze","13":"tretze","14":"catorze","15":"quinze","16":"setze","17" :"disset","18":"divuit","19":"dinou","20" : "vint"}

In [47]:
extKeywords=kdf[kdf['purpose']=='EXTRACTION']

In [48]:
extKeywords['notification_type']=extKeywords['notification_type'].apply(lambda x : x.split('-')[1])


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [87]:
c=0
fgdf['Numlist']=[[] for _ in range(len(fgdf))]
fgdf['MIN DAYS']=''
c=0
for fi,fr in fgdf.iterrows():
    auto=""
    match=None
    match1=set()
    match2=set()
    for i,r in fdf[(fdf['filetype']=='NOTIFICATION')&(fdf['filename'].isin(fr['files']))].iterrows():
        s= r['text_response']
        if len(s.strip())>0:
            s=unidecode.unidecode('\n'.join(list(filter(None,s.split('\n')))).lower())

            fgdf.loc[fi,'Court']=s.split('\n')[0]
            if 'procurador' in s.lower():
                fgdf.loc[fi,'Solictor']=list(filter(None,s[s.lower().index('procurador')+10:].split('\n')))[0]


            ptype=""

            for x in procedures.values():

                if x.lower() in ' '.join(s.split()):

                    fgdf.loc[fi,'Procedure_Type']=x
                    ptype=x
                    break
            if ptype!="":
                if fr['filegroup']=='INADMISION25':
                            print ' YESSS'
                if 'procedimiento' in s.lower():
                    match=re.search(r'(\d{1,20}/\d{4})',s.split('procedimiento')[1])

                else:
                    s1=''.join(s.split())
                    match1=set(re.findall(r'(\d{1,20}/\d{4})',s1))
                    match2={'/'.join(x.split('/')[1:])for x in re.findall(r'(\d{1,2}/\d{1,2}/\d{4})',s1)}

            if not match is None :
                    auto=match.group(0)
            elif not (match1-match2) is None :
                    auto=list(match1-match2)[0] if len(list(match1-match2))>0 else ''
    if(fr['group_predicted_class']=='N1')or(fr['group_predicted_class']=='N3')or(fr['group_predicted_class']=='N4'):
        for ki,kr in extKeywords[(extKeywords['notification_type'].str.contains('AMOUNT'))&(extKeywords['fileclass']==fr['group_predicted_class'])].iterrows():
            for i,r in fdf[(fdf['filename'].isin(fr['files']))&(fdf['filetype']==kr['filetype'])].iterrows():
                text=''.join(r['text_response'].split())
    #         
                if r['text_response'][:5]!='Error':
                    
                    f=True
                    t=text
                    for k in kr['keyword']:
                        k1=''.join(unidecode.unidecode(k).split()).lower()
                        if  k1 in t:
                            t=t[t.index(k1)+len(k1):]
                        else:
                            f=False
                    k1=' '.join(kr['keyword'][0].split())    
                    if f :
                        tex=' '.join(r['text_response'].split())

                        fgdf.loc[fi,'Amount']=tex[tex.index(k1)+len(k1):].split()[0]
    if((fr['group_predicted_class']=='N9')or(fr['group_predicted_class']=='N10')):
#         for ki,kr in extKeywords[(extKeywords['notification_type'].str.contains('TIME FRAME'))&(extKeywords['fileclass']==fr['group_predicted_class'])].iterrows():
            for i,r in fdf[(fdf['filename'].isin(fr['files']))&(fdf['filetype']=='NOTIFICATION')].iterrows():
                text=' '.join(unidecode.unidecode(r['text_response']).split())
    #           
                nls=[]
                nlc=[]
                if r['text_response'][:5]!='Error':
                    
                    f=True
                    t=text
                    
                    k1='dias'
                    
                    if  k1 in t.lower():
                          
                        nls+=  [t.lower().split()[i-1] for i, x in enumerate(t.lower().split()) if 'dias' in x ]
                    elif  'dies' in t.lower():
                          
                        nlc+=  [t.lower().split()[i-1] for i, x in enumerate(t.lower().split()) if 'dies' in x ]
                    else:
                            f=False
                    if f :
                        min=100
                        ls=[]
                        if len(nls)>0:
                            fgdf.at[fi,'Numlist']=nls
                            ls=nls
                            numbers=numspan
                        elif len(nlc)>0:
                            fgdf.at[fi,'Numlist']=nlc
                            ls=nlc
                            numbers=numcat
                        for num in ls:
                            if num in numbers.values():
                                n=int([x for x in numbers.keys() if numbers[x]==num ][0])
                                if n < min:
                                    min=n
                            elif num in numbers.keys():
                                if int(num) < min:
                                    min=int(num)
                            else:
                                print num
                        if min!=100:
                            fgdf.loc[fi,'Time Frame']=min
                        
    if (fr['group_predicted_class']=='N5')|(fr['group_predicted_class']=='N6')|(fr['group_predicted_class']=='N9'):
        

        for ki,kr in extKeywords[~(extKeywords['notification_type'].str.contains('TIME FRAME'))&(extKeywords['fileclass']==fr['group_predicted_class'])].iterrows():
            for i,r in fdf[(fdf['filename'].isin(fr['files']))&(fdf['filetype']=='NOTIFICATION')].iterrows():
                text=''.join(r['text_response'].split())
    #         
                if r['text_response'][:5]!='Error':
                    
                    f=True
                    t=text
                    for k in kr['keyword']:
                        k1=''.join(unidecode.unidecode(k).split()).lower()
                        if  k1 in t:
                            t=t[t.index(k1)+len(k1):]
                        else:
                            f=False
                    k1=' '.join(kr['keyword'][0].split())    
                    if f :
                        fgdf.loc[fi,fr['group_predicted_class']+'-Extraction']=[dict({kr['notification_type']:kr['keyword']})]
    for i,r in fdf[(fdf['filetype']=='TICKET')&(fdf['filename'].isin(fr['files']))].iterrows():
        
            if r.table_response[:5]!='Error':
                try:
                    js= json.loads(unidecode.unidecode( json.dumps(json.loads(r.table_response), ensure_ascii=False)))
                except Exception as e:
                    js=json.loads(r.table_response)
                table2=json.loads(js['table_2'])
#                 print(r['filename'],js["table_1"][[x for x in js["table_1"].keys() if 'Fecha' in x][0]],table2['1'][[x for x in table2['1'].keys() if 'Fecha' in x][0]])
                c+=1
                fgdf.loc[fi,"Document date"]=js["table_1"][[x for x in js["table_1"].keys() if 'Fecha' in x][0]]
                fgdf.loc[fi,"Stamp date"]=table2['1'][[x for x in table2['1'].keys() if 'Fecha' in x][0]]
#                 print js
                
                
                try:
                    fgdf.loc[fi,'Solictor']=js["table_1"]['Destinatarios']['_value']
                    fgdf.loc[fi,'Court']=js["table_1"]['Remitente'][[x for x in js["table_1"]['Remitente'].keys() if str(x)[:6].lower()=='organo'][0]]

                except Exception as e:
                    c+=1
                auto=""
                try:
                    s=json.loads(unidecode.unidecode(r['table_response']))['table_1']['Datos del mensaje']['Procedimiento destino']

                except Exception as e:
                    s=str( json.loads(r['table_response'])['table_1']['Datos del mensaje'])
        #             print str(e)
                match1=set(re.findall(r'(\d{1,20}/\d{4})',s))
                match2={'/'.join(x.split('/')[1:])for x in re.findall(r'(\d{1,2}/\d{1,2}/\d{4})',s)}
                
                f=False
                for x in procedures.keys():
                    if x.lower() in  s.lower():
                        f=True
                        proced= procedures[x]
                if f:
                    fgdf.loc[fi,'Procedure_Type']=proced
                if not (match1-match2) is None :
                    auto=list(match1-match2)[0]
                    

    fgdf.loc[fi,'Auto']=auto
            
print c
fgdf

In [ ]:
d={k:0 for k in list(set(x for x in fgdf['group_predicted_class'].values.tolist() if x==x))}
c=0
for x in fgdf['group_predicted_class'].values.tolist():
    if x==x:
        d[x]+=1
        c+=1
print d ,c

In [86]:
# fgdf['filegroup'].values.tolist()
fgdf['Keywords']=[[] for _ in range(len(fgdf))]
for i , r in fgdf.iterrows():
    for f in r['files']:
        for k,v in fdf.loc[fdf['filename']==f]['keywords'].values[0].items():
            
            if len(fgdf.loc[i,'Keywords'])>0 and  k in fgdf.loc[i,'Keywords'][0].keys() :
                fgdf.loc[i,'Keywords'][0][k]+=v
            else:
                fgdf.loc[i,'Keywords']=[{k:v}]
fgdf['Keywords']=fgdf['Keywords'].apply(lambda x :x[0] if len(x)>0 else {})


In [54]:
import time

In [56]:
asdf = "asdfasfsfsdf"

In [68]:
hashlib.sha1((asdf+str(time.time())).encode()).hexdigest()

'd93387e492c4d6a45ea54014bfcdc0c78b71ce3c'

In [76]:
decisionclass={'N1':'ADMISSION','N2':'TRANSFER OF LEGAL REPRESENTATION PERMITTED',
           'N3':'ENFORCEMENT ORDERS','N4':'STATEMENT OF PAYMENT',
           'N5':'REJECTED CLAIMS','N6':'REJECTED TRANSFER OF LEGAL REPRESENTATION',
           'N7':'HEARING','N8':'ASSET INQUIRY',
           'N9':'PLACE OF RESIDENCY REQUIRED',
           'N10':'REQUIREMENT'}
for i, r in fgdf.iterrows():
        if r['group_predicted_class']==r['group_predicted_class'] and r['group_predicted_class']!='N2+N4':
            fgdf.loc[i,'decision(Actual)']=r['group_predicted_class']+' '+decisionclass[r['group_predicted_class']]
        elif r['group_predicted_class']=='N2+N4':
            fgdf.loc[i,'decision(Actual)']='N2 '+decisionclass['N2']+'+'+'N4 '+decisionclass['N4']
        else:
            fgdf.loc[i,'decision(Actual)']=''
        fgdf.loc[i,'Court(Actual)']=r['Court']
        fgdf.loc[i,'Solictor(Actual)']=r['Solictor']
        fgdf.loc[i,'Document date(Actual)']=r['Document date']
        fgdf.loc[i,'Stamp date(Actual)']=r['Stamp date']
        fgdf.loc[i,'Procedure_Type(Actual)']=r['Procedure_Type']
        fgdf.loc[i,'Amount(Actual)']=r['Amount']
        fgdf.loc[i,'Auto(Actual)']=r['Auto']
        fgdf.loc[i,'Time Frame(Actual)']=r['MIN DAYS']

In [77]:
a = {'fg':fgdf}
with open('/home/thrymr/Racmo/pro/TestFileGroup.pickle', 'wb') as handle:
    pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:

fgdf.loc[fgdf['filegroup']==fgr,'group_predicted_class(Actual)']
fgdf.loc[fgdf['filegroup']==fgr,'Court(Actual)']
fgdf.loc[fgdf['filegroup']==fgr,'Solictor(Actual)']
fgdf.loc[fgdf['filegroup']==fgr,'Document date(Actual)']
fgdf.loc[fgdf['filegroup']==fgr,'Stamp date(Actual)']
fgdf.loc[fgdf['filegroup']==fgr,'Procedure_Type(Actual)']
fgdf.loc[fgdf['filegroup']==fgr,'Amount(Actual)']
fgdf.loc[fgdf['filegroup']==fgr,'Auto(Actual)']
fgdf.loc[fgdf['filegroup']==fgr,'Time Frame(Actual)']


In [ ]:
UPLOAD_FOLDER = "/home/thrymr/Racmo/pro"

with open(UPLOAD_FOLDER+'/TestFileGroup.pickle', 'rb') as handle:
    a=pickle.load(handle)
fgdf=a['fg']

In [ ]:
fdf

In [ ]:
writer = pd.ExcelWriter('/home/thrymr/RACMOIT_File_Group_Result.xlsx')
fgdf.to_excel(writer,'Sheet1')
writer.save()